# import libraries

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns

### install kaggle to use its api for getting data

In [9]:
!pip install kaggle


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### add kaggle.json in your drive for kaggle api access

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
!mkdir data

In [ ]:
import os

os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive/kaggle'

In [ ]:
!kaggle datasets download -d blastchar/telco-customer-churn -p ./data/

In [ ]:
!unzip ./data/telco-customer-churn.zip -d ./data/

In [ ]:
import pandas as pd

df = pd.read_csv('./data/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [ ]:

# Check for null values
print("Number of null values in each column:\n", df.isnull().sum())

# Remove rows with null values
df = df.dropna()

# Perform exploratory data analysis (EDA)
# TODO: add EDA code here

# Preprocess the data
# Split the data into features and target
X = df.drop('Churn', axis=1)
y = df['Churn']

# Handle missing values
# TODO: add code to handle missing values here

# Encode categorical variables
# TODO: add code to encode categorical variables here

# Scale the numerical features
numerical_cols = X.select_dtypes(include=['float64', 'int64']).columns
ct = ColumnTransformer([
        ('scale', StandardScaler(), numerical_cols)], remainder='passthrough')
X = ct.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build binary classification models using Logistic Regression, Decision Tree, and Random Forest
models = [
    ('Logistic Regression', Pipeline([('classifier', LogisticRegression())])),
    ('Decision Tree', Pipeline([('classifier', DecisionTreeClassifier())])),
    ('Random Forest', Pipeline([('classifier', RandomForestClassifier())]))
]

# Define the hyperparameters to search over for each model
param_grids = [
    {
        'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100],
        'classifier__penalty': ['l1', 'l2', 'elasticnet', 'none']
    },
    {
        'classifier__max_depth': [3, 5, 7, 9],
        'classifier__min_samples_split': [2, 5, 10],
        'classifier__min_samples_leaf': [1, 2, 4]
    },
    {
        'classifier__n_estimators': [50, 100, 200],
        'classifier__max_depth': [3, 5, 7, 9],
        'classifier__min_samples_split': [2, 5, 10],
        'classifier__min_samples_leaf': [1, 2, 4]
    }
]

# Train and evaluate each model using grid search
for model, param_grid in zip(models, param_grids):
    clf = GridSearchCV(model[1], param_grid, cv=5)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    fpr, tpr, thresholds = roc_curve(y_test, y_pred)
    auc_score = roc_auc_score(y_test, y_pred)
    
    # Print the results for each model
    print(model[0])
    print("Best hyperparameters: ", clf.best_params_)
    print("Accuracy:", accuracy)
    print("Confusion Matrix:\n", conf_matrix)
    print("AUC Score:", auc_score)
    
    # Visualize the model's performance using a confusion matrix and ROC curve
    plt.figure(figsize=(8,6))
    plt.title('Confusion Matrix')
    sns.heatmap(conf_matrix, annot=True, cmap='Blues')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.show()

    plt.figure(figsize=(8,6))
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % auc_score)
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.show()


Load the dataset into a Pandas dataframe.

Perform exploratory data analysis (EDA) to gain insights into the data.

Preprocess the data by handling missing values, encoding categorical 
variables, and scaling the numerical features.

Split the data into training and testing sets.

Build a binary classification model using scikit-learn.

Tune the hyperparameters of the model using grid search or random search.

Evaluate the model's performance on the test set using appropriate evaluation metrics.

Visualize the model's performance using a confusion matrix and/or ROC curve.